In [118]:
import tensorflow as tf
import zipfile
import os 
import random
import csv
import math
import shutil
from shutil import copyfile

In [12]:
local_zip = './data/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./data')
zip_ref.close()

KeyboardInterrupt: 

In [102]:
tumor=[]
for j in range (1,8):
    data=[]
    img=[]
    with open('./data/GroundTruth.csv') as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)
        for row in reader:
            img.append(row[0] + '.jpg')
            data.append(math.ceil(float(row[j])))
        tumor.append(data)
    print(f"there is {len(data)} data")
print(len(tumor))

there is 10015 data
there is 10015 data
there is 10015 data
there is 10015 data
there is 10015 data
there is 10015 data
there is 10015 data
7


In [123]:
root_dir = "./data/cancerimages/"

if os.path.exists(root_dir):
    shutil.rmtree(root_dir)
    
def create_dir(root_path):
    os.makedirs(os.path.join(root_path, 'training'))
    os.makedirs(os.path.join(root_path, 'validation'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'melanoma'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'melanoma'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'melanocytic nevi'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'melanocytic nevi'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'basal cell carcinoma'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'basal cell carcinoma'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'bowen disease'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'bowen disease'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'benign keratosis'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'benign keratosis'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'dermatofibroma'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'dermatofibroma'))
    
    os.makedirs(os.path.join(f'{root_path}/training', 'vascular lesions'))
    os.makedirs(os.path.join(f'{root_path}/validation', 'vascular lesions'))

create_dir(root_path=root_dir)

In [115]:
for i in range (len(tumor)):
    labels = []
    for j in range (len(tumor[i])):
        if tumor[i][j] == 1:
            labels.append(img[i])
    print(f"this is tumor {i} {len(labels)}")
    split = round(len(labels) * 0.8)

    training = labels[:split]
    val = labels[split:]
    print(len(training))
    print(len(val))

this is tumor 0 1113
890
223
this is tumor 1 6705
5364
1341
this is tumor 2 514
411
103
this is tumor 3 327
262
65
this is tumor 4 1099
879
220
this is tumor 5 115
92
23
this is tumor 6 142
114
28


In [91]:

root = './data/images/'
for i in labels:
    source_img = os.path.join(root, i)
    source_path_melanomas = os.path.join(source_path_melanoma, i)
    copyfile(source_img, source_path_melanomas)

directory already made


In [92]:
print(f"this file have {len(os.listdir(source_path_melanoma))}")

this file have 1113
